## Project 1, tornadoes and mobile home parks in the US, risk analysis.

In [1]:
# import dependencies

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


In [3]:
# define paths for both files

tornadoes_data_path = "1950-2021_all_tornadoes.csv"
mobile_home_parks_path = "Mobile_Home_Parks.csv"

# Read the data from both csv's 

tornadoes_data = pd.read_csv(tornadoes_data_path)
mobile_home_data = pd.read_csv(mobile_home_parks_path)

display(tornadoes_data)
display(mobile_home_data) 


,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
0,192,1950,10,1,1950-10-01,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
1,193,1950,10,9,1950-10-09,02:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
2,195,1950,11,20,1950-11-20,02:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
3,196,1950,11,20,1950-11-20,04:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
4,197,1950,11,20,1950-11-20,07:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68863,620523,2021,9,8,2021-09-08,15:32:00,3,PA,42,0,...,0.73,30,1,1,1,109,0,0,0,0
68864,620524,2021,9,8,2021-09-08,15:35:00,3,PA,42,0,...,1.45,30,1,1,1,109,0,0,0,0
68865,620525,2021,9,8,2021-09-08,19:08:00,3,FL,12,0,...,1.45,50,1,1,1,129,0,0,0,0
68866,620526,2021,9,9,2021-09-09,02:10:00,3,CT,9,0,...,0.75,75,1,1,1,13,0,0,0,0


,index,X,Y,FID,MHPID,NAME,ADDRESS,ADDRESS2,CITY,STATE,...,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,UNITS,SIZE,YEARBLT,RevGeoFlag
0,0,-81.687714,28.852114,2001,2101,Sharp's Mobile Park,43 Sharp Circle,Not Available,Eustis,FL,...,Residential trailer parks,Open,1900-01-01T00:00:00.000Z,Both Internet and Imagery,2013-03-04T00:00:00.000Z,Not Available,0,Not Available,,No
1,1,-85.232052,30.771086,2002,2001,Stage Creek Trailer Park,4366 Pearl Street,Not Available,Marianna,FL,...,Residential trailer parks,Open,1900-01-01T00:00:00.000Z,Imagery or Map Only,2013-03-01T00:00:00.000Z,Not Available,0,Not Available,,No
2,2,-85.163627,30.789739,2003,2002,Springside Estates,5164 Copper Lane,Not Available,Marianna,FL,...,Residential trailer parks,Open,1900-01-01T00:00:00.000Z,Imagery or Map Only,2013-03-01T00:00:00.000Z,Not Available,0,Not Available,,No
3,3,-81.687733,28.815828,2004,2102,Lake Saunders Mobile Home Park,4301 Hwy 19A,Not Available,Mount Dora,FL,...,Mobile (manufactured) home parks,Open,1900-01-01T00:00:00.000Z,Imagery or Map Only,2013-03-06T00:00:00.000Z,Not Available,0,Not Available,,No
4,4,-81.784127,28.529614,2005,2103,Jones Cypress Cove Resort,10233 Cypress Cove Lane,Not Available,Clermont,FL,...,Mobile (manufactured) home parks,Open,1900-01-01T00:00:00.000Z,Imagery or Map Only,2013-03-06T00:00:00.000Z,Not Available,0,Not Available,,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38524,38524,-92.391481,34.884523,38525,41433,Not Available,14121 Overstreet Rd,Not Available,Maumelle,AR,...,Residential trailer parks,Imagery,2015-01-30T00:00:00.000Z,Both Internet and Imagery,2015-01-30T00:00:00.000Z,Not Available,0,S,,No
38525,38525,-84.350860,40.832630,38526,41219,Not Available,826 Skinner St,Not Available,Delphos,OH,...,Residential trailer parks,Imagery,2015-01-23T00:00:00.000Z,Both Internet and Imagery,2015-01-23T00:00:00.000Z,Not Available,0,M,,No
38526,38526,-83.823340,40.754216,38527,41220,Not Available,4525 State Route 235,Not Available,Ada,OH,...,Recreational vehicle parks,Imagery,2015-01-23T00:00:00.000Z,Both Internet and Imagery,2015-01-23T00:00:00.000Z,Not Available,0,M,,No
38527,38527,-83.815211,40.769507,38528,41221,Not Available,547 E Lincoln Ave,Not Available,Ada,OH,...,Residential trailer parks,Imagery,2015-01-23T00:00:00.000Z,Both Internet and Imagery,2015-01-23T00:00:00.000Z,Not Available,0,M,,No
